# **PROYECTO FINAL** ╰(*°▽°*)╯ 
### **Generación de Portafolio Dinámico usando DCC GARCH**
**Mercado de capitales** - **Alexander Chi Fernandez, Alejandro César**

***Introducción:***

Si la correlación entre los activos varían a lo largo del tiempo, utilizar un promedio histórico de correlaciones para construir el portafolio no garantiza optimalidad. En ese caso, los pesos obtenidos **no minimizan la varianza ni maximizan el ratio de Sharpe** (dependiendo la estrategia) en cada periodo 𝑡, ya que no reflejan las dependencias dinámicas entre los activos.

Para capturar la correlación (o covarianza) dinámica y condicional entre los activos, se utiliza el modelo **DCC-GARCH**. Este modelo genera una matriz de covarianza condicional para cada periodo 𝑡, la cual permite actualizar los pesos del portafolio de forma dinámica, adaptándose a los cambios en la relación entre los activos.

Nuestra estrategia consiste en rebalancear el portafolio con una frecuencia mensual. Los pesos se ajustan en función del promedio mensual de las matrices de covarianza y del promedio de los rendimientos de cada activo, calculados con una ventana móvil de 20 días. El rebalanceo se realiza al final de cada mes, adaptando así la composición del portafolio a las condiciones más recientes del mercado.

### **Definición del modelo: DCC GARCH** 🤠

El modelo **DCC-GARCH (Dynamic Conditional Correlation GARCH)**, propuesto por *Engle (2002)*, permite modelar **varianzas y covarianzas condicionales que varían en el tiempo**, separando la dinámica de **volatilidades individuales** y **correlaciones entre activos**. Esto lo hace especialmente útil para portafolios dinámicos, gestión de riesgo, VaR y pronósticos financieros.

Sea el vector de rendimientos:

$$
r_t = (r_{1t}, r_{2t}, \dots, r_{Nt})'
$$

con esperanza condicional cero y matriz de covarianza condicional:

$$
H_t = \mathbb{E}(r_t r_t' \mid \mathcal{I}_{t-1})
$$

El modelo DCC-GARCH se construye mediante la representación:

$$
r_t = H_t^{1/2} z_t, \qquad z_t \sim (0, I_N)
$$

donde la matriz de covarianza condicional $H_t$ se descompone como:

$$
H_t = D_t R_t D_t
$$

---

### **1. Matriz $D_t$: volatilidades condicionales individuales**

$$
D_t = \mathrm{diag}(\sqrt{h_{1t}}, \sqrt{h_{2t}}, \dots, \sqrt{h_{Nt}})
$$

- Es una **matriz diagonal** que contiene las **desviaciones estándar condicionales** de cada serie.
- Cada $h_{it}$ proviene de un modelo **GARCH(1,1)** univariado aplicado por separado a cada activo.
- Esta matriz captura únicamente la **volatilidad individual** de cada retorno, sin considerar correlaciones entre ellos.

Cada serie sigue:

$$
r_{i,t} = \mu_i + \sigma_{i,t} z_{i,t}, \qquad z_{i,t} \sim t_\nu(0,1)
$$

con la dinámica de volatilidad:

$$
\sigma_{i,t}^2 = \omega_i + \alpha_i r_{i,t-1}^2 + \beta_i \sigma_{i,t-1}^2
$$

donde:
- $\alpha_i$ mide la **sensibilidad a choques recientes**,
- $\beta_i$ mide la **persistencia de la volatilidad**,
- la distribución *t-Student* permite capturar **colas pesadas**, típicas en retornos financieros.
- Los parametros se estiman utilizando Máxima Verosimilitud (MLE)
---

### **2. Matriz $Q_t$: correlación condicional no normalizada**

La dinámica DCC comienza con:

$$
Q_t = (1 - a - b)\bar{Q} + a (z_{t-1} z_{t-1}') + b Q_{t-1}
$$

donde:

- $z_t = D_t^{-1} r_t$ son los **residuos estandarizados**, con varianza unitaria,
- $Q_t$ es una **matriz de covarianza condicional** de estos residuos estandarizados.

**Importante:**  
$Q_t$ **todavía no es una matriz de correlación válida**, ya que:
- su diagonal no necesariamente es igual a 1,
- puede contener valores fuera de $[-1, 1]$,
- aún no está escalada correctamente.

Por eso se le llama **correlación no normalizada** o "matriz cruda" del proceso DCC.

- $\bar Q$ es la **matriz de covarianza incondicional** de los residuos estandarizados, y actúa como el “nivel de largo plazo”.
- Los parámetros $a$ y $b$ cumplen:
  - $a \ge 0$: respuesta inmediata de la correlación a un choque,
  - $b \ge 0$: persistencia de la correlación,
  - $a + b < 1$: condición de estabilidad del proceso DCC.
  - Los parametros se estiman utilizando Máxima Verosimilitud (MLE)

---

### **3. Matriz $R_t$: correlación condicional dinámica**

La matriz de correlación válida se obtiene normalizando $Q_t$:

$$
R_t = \mathrm{diag}(Q_t)^{-1/2} \, Q_t \, \mathrm{diag}(Q_t)^{-1/2}
$$

Esta transformación:
- fija la diagonal de $R_t$ en uno,
- restringe los elementos a $[-1,1]$,
- asegura que $R_t$ sea **definida positiva**,
- convierte a $Q_t$ en una **matriz de correlaciones condicionales** propiamente dicha.

Cada elemento $R_{ij,t}$ representa la **correlación dinámica condicional** entre los activos $i$ y $j$ en el tiempo $t$.

---

### **4. Matriz $H_t$: covarianza condicional completa**

Una vez obtenidas las volatilidades individuales $D_t$ y la correlación condicional dinámica $R_t$, se reconstruye:

$$
H_t = D_t R_t D_t
$$

- $H_t$ combina tanto la **volatilidad individual** como la **dependencia dinámica** entre activos.
- Proporciona la **matriz de covarianza condicional completa**

### **Portafolio dinámico** 🤠

El rendimiento del portafolio en el periodo $t$ está definido por:

$$R_{p,t} = \mathbf{w}' \mathbf{r}_t$$

y su varianza condicional por:

$$\mathrm{Var}(R_{p,t} \mid \mathcal{I}_{t-1}) = \mathbf{w}' H_t \mathbf{w}$$

donde:
- $\mathbf{w} = (w_1,\dots,w_N)'$ es el **vector de pesos** del portafolio,
- $\mathbf{r}_t$ es el **vector de rendimientos**,
- $H_t$ es la **matriz de covarianza condicional** proporcionada por el modelo DCC-GARCH.

---

### **1. Uso de la matriz de covarianza DCC para rebalanceo mensual**

Para el rebalanceo **mensual**, usamos el **promedio de las matrices de covarianza del mes**:

$$\bar{H}_{\text{mes}} = \frac{1}{|\mathcal{T}_{\text{mes}}|}\sum_{t\in\mathcal{T}_{\text{mes}}} H_t$$

- $\mathcal{T}_{\text{mes}}$ es el conjunto de días hábiles del mes.
- $\bar{H}_{\text{mes}}$ suaviza la volatilidad diaria y da estabilidad al rebalanceo.

---

### **2. Estimación de la esperanza de rendimientos**

Para la expectativa de rendimientos $\boldsymbol{\mu}$ usamos una **ventana móvil de 20 días**:

$$\hat{\mu}_{t} = \frac{1}{20}\sum_{s=t-19}^{t} \mathbf{r}_s$$

- La $\hat{\mu}_t$ usada en el rebalanceo mensual se toma del **último día hábil previo al rebalanceo**.

---

### **3. Problema de optimización: maximizar Sharpe**

Se busca maximizar el **Sharpe ratio**:

$$\max_{\mathbf{w}} \; \text{Sharpe}(\mathbf{w})$$

sujeto a:

$$\mathbf{1}'\mathbf{w} = 1,\quad \mathbf{w}\ge0$$

y con $r_f = 0$:

$$\text{Sharpe}(\mathbf{w}) = \frac{\mathbf{w}'\hat{\mu}}{\sqrt{\mathbf{w}'\bar{H}_{\text{mes}}\mathbf{w}}}$$

La optimización se llava a cabo del métoro numérico SLSQP (Sequential Least Squares Programming).

### **Conclusión**
El enfoque de portafolio dinámico basado en el modelo DCC-GARCH permite capturar de manera realista la naturaleza cambiante del riesgo y la dependencia entre los activos financieros. A través de la estimación diaria de la matriz de covarianza condicional $H_t$, y su posterior agregación mensual mediante $\bar{H}_{\text{mes}}$, se obtiene una medida de riesgo más estable y representativa para las decisiones de inversión. Al combinar esta matriz con una estimación adaptativa de la expectativa de rendimientos —calculada mediante una ventana móvil de 20 días— se logra un sistema de optimización que responde de forma flexible a las condiciones del mercado.

En conjunto, este marco genera un portafolio capaz de ajustarse dinámicamente a los cambios en volatilidad y correlaciones, optimizando de manera consistente el Sharpe ratio bajo restricciones realistas. El resultado es una estrategia robusta, coherente con la teoría moderna de portafolios y respaldada por un modelo econométrico sólido que aprovecha los patrones temporales de la volatilidad y la dependencia entre activos.

## **Librerías** ✅

*Descripción:*

Librerías a utilizar para el proyecto.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
import plotly.graph_objects as go
import plotly.colors as pc
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.optimize import minimize
import plotly.express as px
from scipy.optimize import fmin, minimize
from IPython.display import display
from scipy.stats import t, norm
import warnings
import pyreadr
import xarray as xr


## **Funciones generales** ✅

*Descripción:*

Fucniones personalizadas para facilitar y evitar los procesos repetitivos.

### DCC GARCH

In [ ]:
def below_diagonal(arr):
    """
    Extrae todos los elementos por debajo de la diagonal (i > j)
    de un array 3D de shape (n, n, T) y regresa un DataFrame
    con una columna por cada par (i,j).

    Parameters
    ----------
    arr : numpy.ndarray
        Array 3D con dimensiones (n, n, T)

    Returns
    -------
    pandas.DataFrame
        DataFrame con dimensiones T x [n(n-1)/2]
        Columnas con nombres "i_j" (1-indexado)
    """
    
    if arr.ndim != 3:
        raise ValueError("El array debe tener 3 dimensiones (n, n, T).")
    
    n, n2, T = arr.shape
    if n != n2:
        raise ValueError("El array debe ser cuadrado en sus dos primeras dimensiones.")
    
    data = {}

    for i in range(n):
        for j in range(i):
            colname = f"{i+1}_{j+1}"  # pares 1-indexados
            data[colname] = arr[i, j, :]  # serie temporal del elemento (i,j)

    return pd.DataFrame(data)


### Portafolio

In [ ]:
def rebalancear_portafolio(dfr, cov_dcc, port_spec, frecuencia="monthly", ventana=20, objetivo="sharpe"):
    """
    Función para rebalancear un portafolio según diferentes frecuencias y objetivos
    
    Parameters:
    -----------
    dfr : pandas DataFrame
        DataFrame con los retornos de los activos
    cov_dcc : numpy array 3D
        Array 3D con matrices de covarianza DCC [assets x assets x time]
    port_spec : dict
        Especificaciones del portafolio (restricciones, etc.)
    frecuencia : str
        Frecuencia de rebalanceo ("daily", "weekly", "monthly", "quarterly", "yearly")
    ventana : int
        Ventana móvil para cálculo de medias
    objetivo : str
        Objetivo de optimización ("sharpe" o "var")
    
    Returns:
    --------
    pandas DataFrame con los pesos diarios del portafolio
    """
    
    # Validaciones iniciales
    if objetivo not in ["sharpe", "var"]:
        raise ValueError("Objetivo inválido. Usa: 'sharpe' o 'var'")
    
    if frecuencia not in ["daily", "weekly", "monthly", "quarterly", "yearly"]:
        raise ValueError("Frecuencia inválida. Usa: 'daily', 'weekly', 'monthly', 'quarterly' o 'yearly'")
    
    # Definir puntos de rebalanceo según frecuencia
    end_points = _get_rebalance_points(dfr.index, frecuencia)
    
    # Crear DataFrame para guardar los pesos diarios
    weights_df = pd.DataFrame(
        index=dfr.index,
        columns=dfr.columns,
        data=np.nan
    )
    
    # Bucle de rebalanceo
    for i, t in enumerate(end_points):
        # Evitar errores fuera de rango
        if t >= cov_dcc.shape[2] or t < ventana:
            continue
        
        try:
            # Media condicional (ventana móvil)
            start_idx = max(0, t - ventana)
            mu_t = dfr.iloc[start_idx:t].mean().values
            
            # Matriz de covarianzas condicionales
            if i == 0:
                cov_start = 0
            else:
                cov_start = end_points[i-1]
            cov_end = t
            
            # Asegurarse de que los índices estén dentro del rango
            cov_start = max(0, min(cov_start, cov_dcc.shape[2] - 1))
            cov_end = max(0, min(cov_end, cov_dcc.shape[2] - 1))
            
            if cov_start > cov_end:
                cov_start = cov_end
                
            cov_slice = cov_dcc[:, :, cov_start:cov_end+1]
            cov_t = np.mean(cov_slice, axis=2)
            
            if cov_t.ndim != 2:
                continue
                
            # Optimización según objetivo
            if objetivo == "var":
                w_t = _optimize_var(mu_t, cov_t, port_spec)
            else:  # sharpe
                w_t = _optimize_sharpe(mu_t, cov_t, port_spec)
            
            # Asignar pesos desde este punto hasta el siguiente
            if i == 0:
                weight_start = 0
            else:
                weight_start = end_points[i-1]
                
            if i == len(end_points) - 1:
                weight_end = len(dfr)
            else:
                weight_end = end_points[i]
            
            # Asegurarse de que los índices estén dentro del rango
            weight_start = max(0, min(weight_start, len(dfr) - 1))
            weight_end = max(0, min(weight_end, len(dfr)))
            
            if weight_start < weight_end:
                weights_df.iloc[weight_start:weight_end] = w_t
            
        except Exception as e:
            warnings.warn(f"Error en optimización en punto {t}: {str(e)}")
            continue
    
    return weights_df

def _get_rebalance_points(dates, frecuencia):
    """
    Obtiene los puntos de rebalanceo según la frecuencia
    """
    if frecuencia == "daily":
        return list(range(len(dates)))
    
    # Convertir a DataFrame temporal para facilitar las operaciones
    temp_df = pd.DataFrame(index=dates)
    
    if frecuencia == "weekly":
        # Último día de cada semana (asumiendo que los datos son diarios)
        temp_df['week'] = temp_df.index.isocalendar().week
        temp_df['year'] = temp_df.index.year
        rebalance_dates = temp_df.groupby(['year', 'week']).tail(1).index
    elif frecuencia == "monthly":
        # Último día de cada mes
        temp_df['year'] = temp_df.index.year
        temp_df['month'] = temp_df.index.month
        rebalance_dates = temp_df.groupby(['year', 'month']).tail(1).index
    elif frecuencia == "quarterly":
        # Último día de cada trimestre
        temp_df['quarter'] = temp_df.index.quarter
        temp_df['year'] = temp_df.index.year
        rebalance_dates = temp_df.groupby(['year', 'quarter']).tail(1).index
    elif frecuencia == "yearly":
        # Último día de cada año
        rebalance_dates = temp_df.groupby(temp_df.index.year).tail(1).index
    
    # Convertir fechas a índices numéricos
    rebalance_indices = [dates.get_loc(date) for date in rebalance_dates]
    
    return sorted(rebalance_indices)

def _optimize_sharpe(mu, cov, port_spec):
    """
    Optimiza para máximo ratio Sharpe
    """
    n_assets = len(mu)
    
    def objective(weights):
        portfolio_return = np.dot(weights, mu)
        portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
        # Minimizar negativo del Sharpe ratio
        if portfolio_vol > 1e-8:
            return -portfolio_return / portfolio_vol
        else:
            return 1e6  # Penalización alta para volatilidad cero
    
    return _optimize_portfolio(objective, n_assets, port_spec)

def _optimize_var(mu, cov, port_spec):
    """
    Optimiza para mínima varianza
    """
    n_assets = len(mu)
    
    def objective(weights):
        return np.dot(weights.T, np.dot(cov, weights))
    
    return _optimize_portfolio(objective, n_assets, port_spec)

def _optimize_portfolio(objective, n_assets, port_spec):
    """
    Función genérica de optimización con restricciones
    """
    # Restricciones básicas
    constraints = []
    
    # Restricción de suma de pesos = 1
    constraints.append({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    # Límites de pesos (ajustar según port_spec)
    bounds = [(0.0, 1.0) for _ in range(n_assets)]  # Largo solamente
    
    # Peso inicial igualmente distribuido
    x0 = np.ones(n_assets) / n_assets
    
    # Optimización
    try:
        result = minimize(
            objective,
            x0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'ftol': 1e-9, 'disp': False, 'maxiter': 1000}
        )
        
        if result.success:
            # Normalizar los pesos para asegurar que sumen 1
            weights = result.x
            weights = np.maximum(weights, 0)  # Eliminar pesos negativos
            weights = weights / np.sum(weights)  # Normalizar
            return weights
        else:
            raise RuntimeError("Optimización no convergió")
            
    except Exception:
        # Fallback: pesos iguales
        warnings.warn("Optimización falló, usando pesos iguales")
        return np.ones(n_assets) / n_assets


### Mtericas

In [ ]:
# Función de máximo drawdown 
def max_drawdown(returns):
    cum_returns = (1 + returns).cumprod()
    peak = cum_returns.cummax()
    drawdown = (cum_returns - peak) / peak
    return drawdown.min()

# Función de métricas 
def metricas(ret_port):
    """
    Calcula métricas financieras básicas del portafolio.
    """
    ret_port = ret_port.dropna()
    media = ret_port.mean()
    desv = ret_port.std()
    rend_anual = (np.prod(1 + ret_port))**(252 / len(ret_port)) - 1
    vol_anual = desv * np.sqrt(252)
    sharpe = media / desv * np.sqrt(252)
    max_dd = max_drawdown(ret_port)
    
    print("----- Métricas del portafolio -----")
    print(f"Rendimiento diario promedio: {media:.6f}")
    print(f"Desviación estándar diaria: {desv:.6f}")
    print(f"Rendimiento anualizado: {rend_anual:.4f}")
    print(f"Volatilidad anualizada: {vol_anual:.4f}")
    print(f"Ratio de Sharpe (Rf = 0): {sharpe:.4f}")
    print(f"Máximo drawdown: {max_dd:.4f}")

# Función para calcular VaR paramétrico
def VaR(valor, std, v_critico):
    var = pd.DataFrame(valor * std * v_critico * -1)
    var.columns = ['Valor en riesgo']
    return var

### Visualización

In [ ]:
# Función para visualizar
def Visualizar_Series_Dinamica(df, tickers, nombre_de_eje_y='Y', titulo='TITULO'):
    fig = go.Figure()

    # Paleta de colores cualitativos de Plotly (hasta 10 colores distintos)
    colores = pc.qualitative.Plotly

    for i, ticker in enumerate(tickers):
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df[ticker],
            name=ticker,
            line=dict(color=colores[i % len(colores)], width=2)
        ))

    # Layout y estética
    fig.update_layout(
        title=dict(
            text=titulo,
            x=0.5,
            font=dict(size=20, family='Gill Sans', color='black')
        ),
        xaxis=dict(title='Fecha'),
        yaxis=dict(
            title=nombre_de_eje_y,
            titlefont=dict(size=14),
            tickfont=dict(size=12)
        ),
        legend=dict(x=0.5, y=1.1, orientation='h', xanchor='center'),
        margin=dict(t=80),
        template='plotly_white',
        height=500
    )

    fig.show()
    return fig

## **Carga de insumos** ✅

*Descripción:*

Carga de las series de tiempo, el calculo de sus rendimientos (logaritmicos y simples) y sus visualizaciones

### Series financieras

In [ ]:
# --- 1. Descargar insumos ---
# Definir tickers
tickers = ['WMT', 'JNJ', 'NKE', 'CROX']

# Descargar datos
data = yf.download(tickers, start='2023-01-01', end=pd.Timestamp.today().strftime('%Y-%m-%d'))['Close']

C:\Users\Alexander\AppData\Local\Temp\ipykernel_30636\1399197418.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  4 of 4 completed


In [ ]:
# --- 2. Calculo de rendimientos ---
# Calcular rendimientos logarítmicos
dfr = np.log(data).diff().dropna()

# Calcular rendimientos simples
dfrs = data.pct_change().dropna()

In [ ]:
# --- 3. Visualizaciones ---
precios = Visualizar_Series_Dinamica(data, data.columns, nombre_de_eje_y='Precio de Cierre ', titulo='Precios Historicos')
rendimientis_simples = Visualizar_Series_Dinamica(dfrs, dfrs.columns, nombre_de_eje_y='Rendimientos simples ', 
                                                  titulo='Rendimientos Simples Historicos')
rendimientis_log = Visualizar_Series_Dinamica(dfr, dfr.columns, nombre_de_eje_y='Rendimientos logaritmicos', 
                                              titulo='Rendimientos Logaritmicos Historicos')
rendimientis_log2 = Visualizar_Series_Dinamica(dfr**2, dfr.columns, nombre_de_eje_y='Rendimientos cuadráticos', 
                                              titulo='Rendimientos Logaritmicos Cuadráticos Historicos')

### Matrices DCC

In [ ]:
# Leer las matrices cor y cov
cor_result = pyreadr.read_r(r"C:\Users\Alexander\OneDrive\桌面\Proyecto final mercado de capitales\cor_dcc.rds")
cov_result = pyreadr.read_r(r"C:\Users\Alexander\OneDrive\桌面\Proyecto final mercado de capitales\cov_dcc.rds")

# Extraer el único objeto guardado
cor_dcc_xr = cor_result[None]   # xarray.DataArray
cov_dcc_xr = cov_result[None]

# Convertir a numpy
cor_dcc = cor_dcc_xr.values
cov_dcc = cov_dcc_xr.values

## **Ejecución de funciones** ✅

*Descripción:*

En esta sección se realiza 3 bloques:

- DCC: Calcular las matrices de correlación y covarianza, asi mismo mostrar el resultado de DCC.
- Optimización del portafolio: Encontrar los pesos optimos para el portafolio dinámico.
- Volatilidad condicional del portafolio: Modelar la volatilidad condicional del portafolio usando GARCH.

### DCC

In [ ]:
# --- 1. Obtener las correlaciones cruzadas
correlacion_dcc = below_diagonal(cor_dcc)

In [ ]:
# --- 2. Visualización de DCC
# Preparando
stock_names = [x.split()[0] for x in dfr.iloc[:,:dfr.shape[1]].columns] # Nombres de los activos
corr_name_list = [] # Combinaciones de nombres
for i, name_a in enumerate(stock_names):
    if i == 0:
        pass
    else:
        for name_b in stock_names[:i]:
            corr_name_list.append(name_a + "-" + name_b)

correlacion_dcc.columns = corr_name_list
correlacion_dcc.index = dfr.iloc[:,:dfr.shape[1]].dropna().index

# Visualizar
dcc_corr_plot = Visualizar_Series_Dinamica(correlacion_dcc, correlacion_dcc.columns, nombre_de_eje_y='Correlaciones DCC ', titulo='Resultado de DCC GARCH')

### Optimización del Portafolio

In [ ]:
# ---1. Optimización del portafolio
if __name__ == "__main__":
    # Especificaciones del portafolio
    port_spec = {
        'min_weight': 0.0,
        'max_weight': 1.0
    }
    
    # Ejecutar rebalanceo
    try:
        w = rebalancear_portafolio(
            dfr, 
            cov_dcc, 
            port_spec, 
            frecuencia="monthly", 
            objetivo="sharpe"
        )
        
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
# Visualización de los pesos a lo largo del tiemoi
w_plot = Visualizar_Series_Dinamica(w, w.columns, nombre_de_eje_y='Ponderaciones ', titulo='Pesos por activo')

### Ajustar un GARCH al portafolio

In [ ]:
# ---1. Ajustar GARCH(1,1) al retorno logarítmico del portafolio --- 
# Calculo de rendimientos 
# Alinear pesos (usar los del periodo anterior)
w_lag = w.shift(1)

# Rendimientos logarítmicos del portafolio dinámico 
ret_port_log = pd.DataFrame((w_lag * dfr).sum(axis=1))
ret_port_log.columns = ["Rendimiento_log_Portafolio"]

# Ajustar modelo GARCH
gm = arch_model(ret_port_log, mean='Constant', vol='Garch', p=1, q=1, dist='normal')
garch_ret_p = gm.fit()  # Ajuste del modelo

# Volatilidad condicional estimada 
sigma_t = pd.DataFrame(garch_ret_p.conditional_volatility)
sigma_t.columns = ["Desviacion estándar condicional"]

garch_ret_p.summary()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 5103592887.849066
Iteration:      2,   Func. Count:     18,   Neg. LLF: 1882.8791192295328
Iteration:      3,   Func. Count:     28,   Neg. LLF: 9777444.57849563
Iteration:      4,   Func. Count:     40,   Neg. LLF: 2036.7620250017717
Iteration:      5,   Func. Count:     50,   Neg. LLF: 636.0402104418936
Iteration:      6,   Func. Count:     59,   Neg. LLF: 1145.8047205536213
Iteration:      7,   Func. Count:     68,   Neg. LLF: -92.85014938228369
Iteration:      8,   Func. Count:     77,   Neg. LLF: -1781.7363724305533
Iteration:      9,   Func. Count:     83,   Neg. LLF: -1793.3613665055882
Iteration:     10,   Func. Count:     91,   Neg. LLF: -1826.1025191526196
Iteration:     11,   Func. Count:     97,   Neg. LLF: -1881.3703622933758
Iteration:     12,   Func. Count:    103,   Neg. LLF: -1885.5237873572048
Iteration:     13,   Func. Count:    108,   Neg. LLF: -1885.8840308939355
Iteration:     14,   Func. Count:    113,   Neg. 

c:\Users\Alexander\anaconda3\envs\r_env\lib\site-packages\arch\univariate\base.py:694: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003915. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.




<class 'statsmodels.iolib.summary.Summary'>
"""
                         Constant Mean - GARCH Model Results                          
======================================================================================
Dep. Variable:     Rendimiento_log_Portafolio   R-squared:                       0.000
Mean Model:                     Constant Mean   Adj. R-squared:                  0.000
Vol Model:                              GARCH   Log-Likelihood:                1885.92
Distribution:                          Normal   AIC:                          -3763.84
Method:                    Maximum Likelihood   BIC:                          -3745.53
                                                No. Observations:                  719
Date:                        Fri, Nov 14 2025   Df Residuals:                      718
Time:                                20:21:11   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu         2.2984e-03  6.398e-04      3.592  3.276e-04 [1.044e-03,3.552e-03]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      7.1836e-05  1.902e-05      3.778  1.584e-04 [3.456e-05,1.091e-04]
alpha[1]       0.5729      0.222      2.585  9.750e-03     [  0.138,  1.007]
beta[1]        0.4204      0.105      3.988  6.676e-05     [  0.214,  0.627]
============================================================================

Covariance estimator: robust
"""

## **Resultados del portafolio** ✅

*Descripción:*

Se calculan metricas y algunas visualizaciones

In [ ]:
# --- 1. Calculo de rendimientos ---
# Alinear pesos (usar los del periodo anterior)
w_lag = w.shift(1)

# Rendimientos logarítmicos del portafolio dinámico 
ret_port_log = pd.DataFrame((w_lag * dfr).sum(axis=1))
ret_port_log.columns = ["Rendimiento_log_Portafolio"]

#  Rendimientos simples del portafolio dinámico 
ret_port_simple = (w_lag * dfrs).sum(axis=1)
ret_port_simple.columns = ["Rendimiento_simple_Portafolio"]

#  Ajuste por costo de transacción 
costo_tx = 0.0025
#changes = w_lag.diff().abs()
changes = w.diff().abs()

costo_periodo = changes.sum(axis=1) * costo_tx
ret_ajustado = ret_port_simple - costo_periodo
ret_ajustado = ret_ajustado.dropna()

# Rendimiento acumulado
ret_cum = pd.DataFrame((1 + ret_ajustado).cumprod())
ret_cum.columns = ["Rendimiento Acumulado"]

In [ ]:
# --- 2. Valor del portafolio ---
# Valor del portafolio
capital = 10000
valor = capital * (ret_cum)

In [ ]:
# --- 3. Calculo del VaR ---
var = VaR(valor['Rendimiento Acumulado'], sigma_t['Desviacion estándar condicional'], 1.96)

In [ ]:
# --- 4. Métricas del portafolio
metricas(ret_ajustado)

----- Métricas del portafolio -----
Rendimiento diario promedio: 0.002950
Desviación estándar diaria: 0.020310
Rendimiento anualizado: 0.9978
Volatilidad anualizada: 0.3224
Ratio de Sharpe (Rf = 0): 2.3054
Máximo drawdown: -0.1349


In [ ]:
# --- 5. Visualizaciones ---
ret_cum_plot = Visualizar_Series_Dinamica(ret_cum, ret_cum.columns, nombre_de_eje_y='Rendimiento Acumulado', 
                           titulo='Rendimientos acumulados del portafolio dinámico')
volatilidad = pd.concat([sigma_t, ret_port_log], axis=1)
volatilidad_plot = Visualizar_Series_Dinamica(volatilidad, volatilidad.columns, nombre_de_eje_y='Rendimientos logaritmicos / Desviación estándar condicional', titulo='Resultado del GARCH (1,1) aplicado al portafolio dinámico')
var_plot = Visualizar_Series_Dinamica(var, var.columns, nombre_de_eje_y='Capital', titulo='Valor en Riesgo paramétrico a 95%')


## **Comparativa con SPY y un portafolio estatico** ✅

In [ ]:
# --- 1. Descargar insumos ---
# Definir tickers
tickers = ['SPY']

# Descargar datos
data_spy = yf.download(tickers, start='2023-01-01', end=pd.Timestamp.today().strftime('%Y-%m-%d'))['Close']

# --- 2. Calculo de rendimientos ---
# Calcular rendimientos simples
dfrs_spy2 = data_spy.pct_change().dropna()
ret_cum_spy = pd.DataFrame((1 + dfrs_spy2).cumprod())


# --- 3. Visualizaciones ---
data_spy_plot = Visualizar_Series_Dinamica(data_spy, data_spy.columns, nombre_de_eje_y='Precio de Cierre ', titulo='Precios Historicos')

# --- 4. Métricas del portafolio
metricas(dfrs_spy2['SPY'])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_30636\1683691414.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


----- Métricas del portafolio -----
Rendimiento diario promedio: 0.000889
Desviación estándar diaria: 0.009777
Rendimiento anualizado: 0.2360
Volatilidad anualizada: 0.1552
Ratio de Sharpe (Rf = 0): 1.4429
Máximo drawdown: -0.1876


In [ ]:
# --- 4. Visualizaciones ---
ret_cum.columns = ['Portafolio dinámico']
vs = pd.concat([ret_cum, ret_cum_spy], axis=1)
ret_cum_plot = Visualizar_Series_Dinamica(vs, vs.columns, nombre_de_eje_y='Rendimiento Acumulado', 
                           titulo='Comparaciones')